### Dependencies

IMPORTANT XSTeam units are throwing everything off

Units here are Celcius, all power is in KW and all energy in kJ

In [1]:
using ModelingToolkit, Plots, DifferentialEquations, Revise, ModelingToolkitStandardLibrary, Unitful
using Symbolics, Logging
using XSteam

@variables t
Logging.disable_logging(Logging.Warn)
MTK = ModelingToolkit

ModelingToolkit

In [ ]:
@register_symbolic cpfunc(x,y)
@register_symbolic kfunc(x,y)
@register_symbolic vphfunc(x,y)

@register_symbolic Tpsfunc(x,y)
@register_symbolic Tphfunc(x,y)

@register_symbolic xphfunc(x,y)
@register_symbolic xptfunc(x,y)

@register_symbolic sptfunc(x,y)
@register_symbolic sphfunc(x,y)

@register_symbolic hptfunc(x,y)
@register_symbolic hpsfunc(x,y)
@register_symbolic hsatfunc(x)

hydro_prop_dict = Dict(Tpsfunc => T_ps,
                        Tphfunc => T_ph,
                        xphfunc => x_ph,
                        sptfunc => s_pT,
                        sphfunc => s_ph,
                        hptfunc => h_pT,
                        hpsfunc => h_ps,
                        hsatfunc => hL_p,
                        vphfunc => v_ph);


# propDict = Dict(cpfunc => cphe, kfunc => khe)

### Pins

In [ ]:
"""
    BasicSteamPin()
    Self computes T,s,x,V
    Must have methods for ṁ,Φ,P,h
"""
@connector function BasicSteamPin(; name, Pdef=0.1)
    across_var  = @variables  P(t)=Pdef T(t)=190 s(t)=0.0 h(t)=191 x(t)=0.0 v(t)=.001
    thru_var    = @variables  ṁ(t)=0.0 Φ(t)=0.0                     # mass flow and energy flow
    
    ps = []
    
    eqs = Equation[T ~ Tphfunc(P,h)
                    s ~ sphfunc(P,h)
                    x ~ xphfunc(P,h)
                    v ~ vphfunc(P,h)]

    ODESystem(eqs, t, [thru_var..., across_var...], ps; name = name, defaults = [P => Pdef, ṁ => 0, Φ => 0, h=>191])
end

@connector function WorkPin(; name)
    sts = @variables Ẇ(t) = 0.0 
    ODESystem(Equation[], t, sts, []; name = name)
end

@connector function HeatTransferPin(; name)  #input in W
    sts = @variables Q̇(t) = 0.0
    ODESystem(Equation[], t, sts, []; name = name, defaults = [Q̇ => 0.0])
end

@connector function FixedHeatFlowPin(; name, Qin = 1e3)  #input in W
    sts = @variables Q̇(t)=Qin 
    ps = @parameters Qin = Qin
    eqs=[Q̇ ~ Qin]
    ODESystem(eqs,t, sts, ps; name = name, defaults = [Q̇ => Qin])
end

# @named bsp_test = BasicSteamPin()
# p = WorkPin(name = :testwork)
# q = FixedHeatFlowPin(name = :testfixedheatflow)
# ht = HeatTransferPin(name = :testHeatTransferPin)

### Connecting functions

In [ ]:
function hydro_basic_connect(n,p)
    eqs = [
        0 ~ p.ṁ + n.ṁ
        0 ~ p.Φ + n.Φ
        p.h ~ n.h
        p.P ~ n.P
    ]
    return eqs
end

function hydro_prop_connect(n,p)
    eqs = [
        p.h ~ n.h
        p.P ~ n.P]
    return eqs
end

function hydro_series_connect(comp ;  returnmode = :eq)
    eqs =  hydro_basic_connect(comp[1].n,comp[2].p)   # n -> p

    if length(comp)>2
        for i = 2:length(comp)-1
            eqs = vcat(eqs,  hydro_basic_connect(comp[i].n,comp[i+1].p))
        end
    end

    if returnmode == :ode
        return ODESystem(eqs,t; name = :connections)
    end
    
    return eqs
end

function extenda(odevec::Vector{ODESystem})
    if length(odevec)==1
        return odevec[1]
    else
        return extend(odevec[1],extenda(odevec[2:end]))
    end
end

### Ports / Consistent Relations

In [ ]:
@component function HeatTransferPort(; name)
    @named p = HeatTransferPin()
    @named n = HeatTransferPin()
    eqs = [0 ~p.Q̇ + n.Q̇]
    @variables Q̇(t) 
    ODESystem(Equation[],t, [Q̇], []; name = name, systems = [p,n])
end


@component function Reservoir(;name, P = 0.1)
    @named n = BasicSteamPin(Pdef = P)
    ps = @parameters P=P 
    eqs = [
        n.P ~ P
        n.h  ~ hsatfunc(P)
        n.Φ ~ 0
    ]
    ODESystem(eqs, t,[], ps; name = name, systems = [n], defaults = [P => 0.1])
end

@component function ioReservoir(;name, P = 0.1)
    @named p = BasicSteamPin(Pdef = 0.1)
    @named n = BasicSteamPin(Pdef = P)
    ps = @parameters P=P
    sts = @variables Φ_in(t) = 0 Pin(t)=P Tin(t)=300 hin(t)=0.0
    eqs = [
        Pin ~ p.P
        Φ_in ~ p.Φ
        Tin ~ p.T
        hin ~ p.h

        0 ~ p.ṁ + n.ṁ

        n.P ~ P
        n.h  ~ hsatfunc(P)
        n.Φ ~ 0
    ]
    ODESystem(eqs, t,sts, ps; name = name, systems = [n,p], defaults = [P => 0.1, Pin => 0.1, Tin => 300, Φ_in => 0.0, hin => 0 ])
end

@component function SetPressure(;name, P = 0.1)
    @named p = BasicSteamPin(Pdef = P)
    @named n = BasicSteamPin(Pdef = P)
    ps = @parameters P(t)=P 
    eqs = [
        p.P ~ P
        n.P ~ p.P
        n.h ~ p.h
        0 ~ p.Φ + n.Φ             # conservation of energy
        0 ~ p.ṁ + n.ṁ
    ]
    ODESystem(eqs, t,[], ps; name = name, systems = [p,n], defaults = [P => 0.1])
end


# @named testres = Reservoir()
# testres.defaults
# s = SteamPort(name = :steamporttes
@named ior = ioReservoir()
states(ior)

### Components

In [ ]:

@component function SteamFlowSource(;name, ṁ = 1.0)
    @named p = BasicSteamPin()
    @named n = BasicSteamPin()
    ps = @parameters Ṁ = ṁ
    eqs = [
            0 ~ p.Φ + n.Φ   # conservation of energy
            0 ~ p.ṁ + n.ṁ
            p.ṁ ~ Ṁ
            0 ~ p.h - n.h 
            n.P ~ p.P
        ]
    ODESystem(eqs, t, [], ps; name = name, systems = [n,p], defaults = [Ṁ => ṁ])
    # compose(ODESystem(eqs, t, [], ps; name = name, defaults = [Ṁ => 1.0]),p,n)
end

@component function AdiabaticPump(;name, η = 0.6, setpressure = false ,Pout = 10)
    #Pout in bar
    @named p = BasicSteamPin()
    @named n = BasicSteamPin()
    @named w = WorkPin()
    
    ps = @parameters η = η P=Pout

    eqs = Equation[
        w.Ẇ ~ p.Φ + n.Φ                         # conservation of energy
        0 ~ p.ṁ + n.ṁ
        n.h  ~ p.h + p.v *100* (n.P - p.P) / η  # work, multiply by 100 to get to kPa
        w.Ẇ  ~ p.ṁ * (n.h - p.h)
    ]

    if setpressure
        eqs = vcat(eqs,n.P  ~ P)
    end
    # extenda([ODESystem(eqs, t,[], ps; name = name, defaults = [η => 0.6, P => Pout]), p,n,w])
    ODESystem(eqs, t,[], ps; name = name, systems = [p,n,w], defaults = [η => .6, P => 10])
end

@component function Splitter(;name)
    @named p = BasicSteamPin()
    @named y = BasicSteamPin()
    @named z = BasicSteamPin()

    #  0 ~ pMax₊P - trbsimo₊split₊z₊P(t)
    eqs = [
        p.h ~ y.h               # enthalpy
        0 ~ y.h - z.h
        p.P ~ y.P               # pressure
        z.P ~ y.P 
        0 ~ p.ṁ + y.ṁ + z.ṁ
        0 ~ p.Φ + y.Φ + z.Φ
    ]

    ODESystem(eqs, t,[], []; name = name, systems = [p,y,z])
end

@component function AdiabaticTurbine(;name, η = 1.0, setpressure = false, Pout = 0.1)
    #Pout in bar
    @named p = BasicSteamPin()
    @named n = BasicSteamPin()
    @named w = WorkPin()
    ps = @parameters P=Pout η = η

    eqs = Equation[
        0 ~ p.ṁ + n.ṁ      
        n.h  ~ p.h - (p.h-hpsfunc(n.P,p.s))*η
        w.Ẇ  ~ p.ṁ * (n.h - p.h)
        w.Ẇ ~ p.Φ + n.Φ                         # conservation of energy
    ]

    if setpressure
        println("we be setting pressure in this bitch")
        eqs = vcat(eqs, n.P ~ P)
    end

    # extenda([ODESystem(eqs, t,[], ps; name = name, defaults = [η => 0.6, P => Pout]), p,n,w])
    ODESystem(eqs, t,[], ps; name = name, systems = [p,n,w], defaults = [η => 1.0, P => 10])
end   

#  0 ~ pMax₊P - trbsimo₊split₊z₊P(t)

@component function SIMO_AdiabaticTurbine(;name, η = 1.0, setpressure = false, Pout = [12,0.1])
    #Pout in bar
    ps = @parameters Py=Pout[1] Pz=Pout[2] η = η

    sp = setpressure

    @named p = BasicSteamPin()
    @named split = Splitter()
    @named hp = AdiabaticTurbine(η = η, Pout = Py, setpressure = sp)
    @named lp = AdiabaticTurbine(η = η, Pout = Pz, setpressure = sp)
    
    sts = @variables Whp(t)=0.0 Wlp(t)=0.0

    eqs = Equation[
        Whp ~ hp.w.Ẇ
        Wlp ~ lp.w.Ẇ
        ]

    # connections

    incon = hydro_basic_connect(p,split.p)
    ycon = hydro_basic_connect(split.y,hp.p)
    zcon = hydro_basic_connect(split.z,lp.p)

    eqs = vcat(eqs,incon,ycon,zcon)

    ODESystem(eqs, t,sts, ps; name = name, systems = [p,split,hp,lp], defaults = [η => 1.0 Py => 10 Pz => 0.1])
    # extend(ODESystem(eqs, t,sts, ps; name = name, systems = [hp,lp], defaults = [η => 1.0 Py => 10 Pz => 0.1]),split)
end

@component function IdealBoiler(;name, Tout = 350)
    #Pout in bar
    @named p = BasicSteamPin()
    @named n = BasicSteamPin()
    @named q = HeatTransferPin()
    
    ps = @parameters T = Tout

    eqs = Equation[
        q.Q̇ ~ p.Φ + n.Φ             # conservation of energy
        0 ~ p.ṁ + n.ṁ
        n.P  ~  p.P                 # no pressure
        n.h  ~ hptfunc(p.P,T)       # work, multiply by 100 to get to kPa
        q.Q̇ ~ p.ṁ * (n.h - p.h)
    ]
    ODESystem(eqs, t,[], ps; name = name, systems = [p,n,q], defaults = [ T => Tout])
end

@component function IdealCondensor(;name)
    #Pout in bar
    @named p = BasicSteamPin()
    @named n = BasicSteamPin()
    @named q = HeatTransferPin()
    
    eqs = Equation[
        q.Q̇ ~ p.Φ + n.Φ             # conservation of energy
        0 ~ p.ṁ + n.ṁ
        n.P  ~  p.P                 # no pressure
        n.h  ~ hsatfunc(p.P)        # work, multiply by 100 to get to kPa
        q.Q̇ ~ p.ṁ * (n.h - p.h)
    ]
    ODESystem(eqs, t,[], []; name = name, systems = [p,n,q])
end

@component function OpenFeedwaterHeater(;name)
    # flows x and y are the inlets
    @named y = BasicSteamPin()
    @named z = BasicSteamPin()
    @named n = BasicSteamPin()  

    sts=@variables yfrac(t) = 0.5

    eqs  =[
        n.P ~ z.P
        n.P ~ y.P
        n.h ~ yfrac * y.h + (1-yfrac) * z.h
        n.h ~ hsatfunc(n.P)
        n.ṁ ~ y.ṁ/yfrac
        0~n.ṁ + y.ṁ + z.ṁ
        0~n.Φ + y.Φ + z.Φ
    ]
    ODESystem(eqs, t,sts, []; name = name, systems = [y,z,n])
end


# @named sftest = SteamFlowSource()
# # sftest.defaults
# @named pftest = AdiabaticPump()
# # pftest.defaults
# # compose(sftest,pftest)
# @named sisotest = SIMO_AdiabaticTurbine()

### TOLKITIZE

In [ ]:
vlv         = SteamFlowSource(name = :vlv, ṁ = 1)
pmpa        = AdiabaticPump(name = :pmpa, Pout = 12,setpressure = true)
pmpb        = AdiabaticPump(name = :pmpb, Pout = 150,setpressure = true)
boil        = IdealBoiler(name = :boil, Tout = 600)
trbsimo     = SIMO_AdiabaticTurbine(name = :trbsimo, setpressure = false) 
openfw      = OpenFeedwaterHeater(name = :openfw)
cond        = IdealCondensor(name = :cond)

ioRes   = ioReservoir(name = :ioRes, P = 0.1)

# @named pMin = SetPressure(P = .1)
# @named pMiddle = SetPressure(P = 12)
# @named pMax = SetPressure(P = 150)

con2 = hydro_basic_connect(pmpa.n,openfw.z)
con3 = hydro_series_connect([openfw,vlv,pmpb,boil,trbsimo])
con4 = hydro_basic_connect(trbsimo.hp.n,openfw.y)
con5 = hydro_series_connect([trbsimo.lp,cond,ioRes,pmpa])
# con6 = hydro_basic_connect(pMin.n,resB.n)
connections = vcat(con1,con2,con3,con4,con5)

odesys = ODESystem(connections,t; name = :odesys, systems = [vlv,pmpa,pmpb,boil,trbsimo,openfw,cond,ioRes])
model = substitute(odesys,hydro_prop_dict)
(model)
# mode = ODEProblem(model,model.defaults,parameters(model))
# parameters(model)
s = states(model)
# parameters(model)
# parameters(model)
# probfun = ODEFunction(model)
# MTK.moderntoolkitize(model)
# model.defaults
# model.states
# states(model)
# equations(model)
# dump(MTK.get_u0_p(model, model.defaults, parameters(model)))
# s[P]
# states(model)

Testing

In [ ]:
res     = Reservoir(name = :res, P = 0.1)
vlv     = SteamFlowSource(name = :vlv, ṁ = 0.5)
pmp      = AdiabaticPump(name = :pmp, Pout = 10, setpressure = true)
boil    = IdealBoiler(name = :boil, Tout = 350)
trb     = AdiabaticTurbine(name = :trb, η = 0.9, Pout = 0.75,setpressure = true)
cond    = IdealCondensor(name = :cond)

connect = hydro_series_connect([res,vlv,pmp,boil,trb,cond])#; returnmode = :ode)

odesys = ODESystem(connect,t; name = :odesys, systems = [vlv,res,pmp,boil,trb,cond])
model = substitute(odesys,hydro_prop_dict)
sys = structural_simplify(model)

para = [vlv.Ṁ => 1.0
        res.P   => 0.75
        pmp.P  => 29.25
        pmp.η => 1.0
        boil.T => 350
        trb.η => 1.0
        trb.P => 0.75]

tspan = (0.0,1.0)
u0 = sys.defaults
# u0[]
prob = ODEProblem(sys,sys.defaults,tspan,para)
# pf(x) = prob.f.observed(x,prob.u0,prob.p,1.0)
# sys.defaults
# trbsimo    = SIMO_AdiabaticTurbine(name = :trbsimo)

In [ ]:
# using Printf
# for s in states(model)
#     @printf "\t %s = %.2d\n" s prob.f.observed(s, prob.u0, prob.p, 1.0)
# end
# @show pf(res.n.h)
# @show pf(res.n.P)
# @show pf(res.n.s)
# @show pf(res.n.x)
# @show pf(res.n.T)
println("pump inlet")
node = pmp.p
@show pf(node.h)
@show pf(node.P)
@show pf(node.s)
@show pf(node.v)
@show pf(node.x)
@show pf(node.Φ)
@show pf(node.T)
println("---------------------------")
node = pmp.n
println("pump outlet")
@show pf(node.h)
@show pf(node.P)
@show pf(node.s)
@show pf(node.v)
@show pf(node.x)
@show pf(node.Φ)
@show pf(node.T)
println("--------------------------")
println("PUMP WORK")
@show pf(pmp.w.Ẇ)
println("--------------------------")
node = boil.n
println("Boiler outlet")
@show pf(node.h)
@show pf(node.P)
@show pf(node.s)
@show pf(node.v)
@show pf(node.x)
@show pf(node.Φ)
@show pf(node.T)
@show pf(boil.q.Q̇)
println("--------------------------")
println("--------------------------")
# node = cond.n
println("Turbine outlet")
@show pf(node.h)
@show pf(node.P)
@show pf(node.s)
@show pf(node.v)
@show pf(node.x)
@show pf(node.Φ)
@show pf(node.T)
@show pf(trb.w.Ẇ)
# @show prob.f.observed(pmp.w.Ẇ,prob.u0,prob.p,1.0)

In [ ]:


con1 = hydro_series_connect([resA,vlv,pmpa,pMiddle])
con2 = hydro_basic_connect(pMiddle.n,ofw.z)
con3 = hydro_series_connect([ofw,pmpb,pMax,boil,trbsimo])
con4 = hydro_basic_connect(trbsimo.hp.n,ofw.y)
con5 = hydro_series_connect([trbsimo.lp,cond,pMin])
con6 = hydro_basic_connect(pMin.n,resB.n)

connections = vcat(con1,con2,con3,con4,con5,con6)

odesys = ODESystem(connections,t; name = :odesys, systems = [vlv,pmpa,pmpb,boil,trbsimo,ofw,cond,pMin,pMiddle,pMax,resA,resB])
model = substitute(odesys,hydro_prop_dict)


In [ ]:




con1 = hydro_series_connect([ofw,pmpb,pMax,boil,trbsimo])
con2 = hydro_series_connect([trbsimo.lp,cond,vlv,pMin,pmpa,pMiddle])
con3 = hydro_basic_connect(trbsimo.hp.n,ofw.y)
con4 = hydro_basic_connect(pMiddle.n,ofw.z)

connections = vcat(con1,con2,con3,con4)

odesys = ODESystem(connections,t; name = :odesys, systems = [vlv,pmpa,pmpb,boil,trbsimo,ofw,cond,pMin,pMiddle,pMax])
model = substitute(odesys,hydro_prop_dict)


para =[
    vlv.Ṁ => 1.0
    pmpa.η => 1.0
    pmpb.η => 1.0
    boil.T => 550
    trbsimo.hp.η => 1.0
    trbsimo.lp.η => 1.0
    pMin.P => 0.1
    pMiddle.P => 12
    pMax.P => 150
]
sys = structural_simplify(model)
# sys = alias_elimination(model)
# tspan = (0.0,1.0)
# prob = ODAEProblem(model,model.defaults,tspan,para)
# pf(x) = prob.f.observed(x,prob.u0,prob.p,1.0)
# # sys.defaults
# display(substitute(connections,hydro_prop_dict))
# display(substitute(odesys.trbsimo,hydro_prop_dict))

#  0 ~ pMax₊P - trbsimo₊split₊z₊P(t)
# parameters(model)
# equations(trbsimo)
# structural_simplify(model)
# structural_simplify(connections)
# con2
# 
# model = substitute(trbsimo.split,hydro_prop_dict)
# [vlv,pmpa,pmpb,boil,trbsimo,ofw,cond,pMin,pMiddle,pMax]


In [ ]:
# using Printf
# for s in states(model)
#     @printf "\t %s = %.2d\n" s prob.f.observed(s, prob.u0, prob.p, 1.0)
# end
# @show pf(res.n.h)
# @show pf(res.n.P)
# @show pf(res.n.s)
# @show pf(res.n.x)
# @show pf(res.n.T)
println("pump inlet")
node = pmp.p
@show pf(node.h)
@show pf(node.P)
@show pf(node.s)
@show pf(node.v)
@show pf(node.x)
@show pf(node.Φ)
@show pf(node.T)
println("---------------------------")
node = pmp.n
println("pump outlet")
@show pf(node.h)
@show pf(node.P)
@show pf(node.s)
@show pf(node.v)
@show pf(node.x)
@show pf(node.Φ)
@show pf(node.T)
println("--------------------------")
println("PUMP WORK")
@show pf(pmp.w.Ẇ)
println("--------------------------")
node = boil.n
println("Boiler outlet")
@show pf(node.h)
@show pf(node.P)
@show pf(node.s)
@show pf(node.v)
@show pf(node.x)
@show pf(node.Φ)
@show pf(node.T)
@show pf(boil.q.Q̇)
println("--------------------------")
println("--------------------------")
node = trb.n
println("Turbine outlet")
@show pf(node.h)
@show pf(node.P)
@show pf(node.s)
@show pf(node.v)
@show pf(node.x)
@show pf(node.Φ)
@show pf(node.T)
@show pf(trb.w.Ẇ)
# @show prob.f.observed(pmp.w.Ẇ,prob.u0,prob.p,1.0)

In [ ]:

# model.defaults
# # para =[
# #     vlv.Ṁ => 1.0
# #     res.P => 0.1
# # ]
# prob = ODEProblem(sys,sys.defaults,tspan,para)
# parameters(sys)
# # MTK.varmap(model)
# MTK.get_iv(model)
# MTK.get_u0_p(model,Pair[], parameters(model))
# MTK.get
# parameters(model)


In [ ]:

pmp      = AdiabaticPump(name = :pump, Pout = 10)

connect = hydro_series_connect([res,vlv,pmp])
@named model = ODESystem(connect,t; systems = [res,vlv,pmp])
model = substitute(model,hydro_prop_dict)
# sys = structural_simplify(model)
sys = model
tspan = (0.0,1.0)
ODEProblem(model,Pair[],tspan)

In [ ]:

sys = substitute(compose(connect,res,vlv,pmp),hydro_prop_dict)
# simple_sys = alias_elimination(sys)
parameters(simple_sys)

u0 = [
    res.P => 0.1
    vlv.p.ṁ => 10.0
]
tspan = (0.0,1.0)
ODEProblem(simple_sys,simple_sys.defaults,tspan)
# states(simple_sys)


# prob = SteadyStateProblem(simple_sys,u0)



In [ ]:

# # @named odesys = compose(connect,res,vlv,pmp)
# odesys = ODESystem(Equation[],t; name = :ODESYS, systems = [connect,res,vlv,pmp])
# odesys.defaults
# @named sys = ODESystem(equations(odesys),t,states(odesys),parameters(odesys))
# structural_simplify(odesys)
# extend(pmp,vlv)
# sys= substitute(sys,hydro_prop_dict)
# odesys = structural_simplify(odesys)

# ss=alias_elimination(sys)


# parameters(odesys)
# tspan = (0.0,10.0)
# prob = ODEProblem(odesys,odesys.defaults,tspan,params)
# c = compose(valv,res,pmp)
# c.defaults
# # f = SteadyStateProblem(odesys,)
# for sys in odesys.systems
#     @show collect(keys(sys.defaults))
# end
# vlv

In [ ]:

# states(odesys)
# parameters(odesys)
# # odesys.defaults
# states(pmp)
# cc=extend(vlv,pmp)
# states(cc)
# odevec = [res,vlv,pmp,connect]
# model = extenda(odevec)
# model = substitute(model,hydro_prop_dict)
# states(pmp)
# parameters(model)
# display(states(model))
# for s in states(model)
#     println(s)
# odesys
# states(res)
# equations(model)